In [76]:
from influxdb_client import InfluxDBClient
import pandas as pd
import os
from IPython.display import Markdown
import warnings

warnings.filterwarnings("ignore")

# Configura la connessione a InfluxDB
INFLUXDB_URL = "http://localhost:8086"
TOKEN = os.getenv("token")  # Assicurati che il token sia impostato correttamente
ORG = "IotAlarmSystem"
BUCKET = "Prova"

client = InfluxDBClient(url=INFLUXDB_URL, token=TOKEN, org=ORG)
query_api = client.query_api()

# Query per recuperare i dati della pressione dall'ultima settimana
query = f"""
from(bucket: "{BUCKET}")
|> range(start: -7d)
|> filter(fn: (r) => r._measurement == "pressure")
|> filter(fn: (r) => r._field == "value")
|> keep(columns: ["_time", "_value"])
"""

result = query_api.query(query)
data = []

# Converti i risultati in un DataFrame
for table in result:
    for record in table.records:
        data.append({"time": record["_time"], "value": record["_value"]})

df = pd.DataFrame(data)

# Se il DataFrame è vuoto, mostra un messaggio
if df.empty:
    print("Nessun dato trovato in InfluxDB.")
else:
    # Converte la colonna "time" in formato datetime
    df["time"] = pd.to_datetime(df["time"])

    df
    print(df)

    # Filtra solo i momenti in cui il valore è 4095 (utente a letto)
    df_sleep = df[df["value"] == 4095]

    # Calcola la durata del sonno per ogni giorno
    df_sleep["date"] = df_sleep["time"].dt.date  # Estrai la data senza orario
    df_sleep["diff"] = df_sleep["time"].diff()   # Calcola la differenza tra i timestamp
    df_sleep["sleep_period"] = df_sleep["diff"].dt.total_seconds() / 3600  # Converti in ore

    # Somma le ore di sonno per ogni giorno
    sleep_summary = df_sleep.groupby("date")["sleep_period"].sum().reset_index()
    sleep_summary.rename(columns={"sleep_period": "hours_slept"}, inplace=True)



                                 time  value
0    2025-01-30 16:52:48.984856+00:00   4095
1    2025-01-30 16:52:49.011028+00:00   4095
2    2025-01-30 16:52:49.020987+00:00   4095
3    2025-01-30 16:52:49.036445+00:00   4095
4    2025-01-30 16:52:49.057940+00:00   4095
...                               ...    ...
1317 2025-01-30 16:59:43.035502+00:00   4095
1318 2025-01-30 16:59:43.054060+00:00   4095
1319 2025-01-30 16:59:43.074008+00:00   4095
1320 2025-01-30 16:59:43.091770+00:00   4095
1321 2025-01-30 16:59:43.112483+00:00   4095

[1322 rows x 2 columns]


In [64]:

Markdown(sleep_summary.to_markdown(index=False))

| date       |   hours_slept |
|:-----------|--------------:|
| 2025-01-30 |    0.00351251 |